# Unet Model Prediction

 - Produce mask images on test dataset
 - Get foreground pixels for mask images and save as pickle file

## Load Test Dataset

In [1]:
from zipfile import ZipFile
from skimage.io import imread
import tensorflow as tf
import numpy as np

def loadData(filepath):
    X = []   # store images
    
    with ZipFile(filepath, 'r') as zipdata:
        namelist = zipdata.namelist()
        
        # loop through every directory/file
        for file_path in namelist:
            # read image files and save to list
            if '.jpg' in file_path or '.JPG' in file_path:
                # load image into list
                img_bytes = zipdata.open(file_path)
                image = np.array(imread(img_bytes))            # read image byte from zip file
                image = tf.image.resize(image, (128, 128))     # resize image
                image = tf.cast(image, tf.float32) / 255.0     # normalize image
                image = image[None,:,:]
                X.append(image)
    
    # close zip file
    zipdata.close()
    
    # return images/labels list
    return X

In [2]:
# load images into list
X_test = loadData('drive/MyDrive/app/data/Oxford-IIIT Pet test.zip')

## Make Prediction & Get Foreground Pixels

In [3]:
# load saved model
model = tf.keras.models.load_model('drive/MyDrive/app/unet_model.h5')

In [4]:
def create_mask(pred_mask):
  pred_mask = tf.argmax(pred_mask, axis=-1)
  pred_mask = pred_mask[..., tf.newaxis]
  return pred_mask[0]

In [5]:
def get_foreground(segmented_image):
  np_image = segmented_image.numpy()
  foreground_pixels = []
  foreground_value = 0

  # iterate through height
  for y in range(segmented_image.shape[0]):
    # iterate through width
    for x in range(segmented_image.shape[1]):
      # if this is a foreground pixel
      if np_image[y, x] == foreground_value:
        foreground_pixels.append((x, y)) # add it to my list of (x, y) coordinate pairs
  
  return foreground_pixels

In [6]:
predictions = []    # predicted mask of the input image
fg_pixels = []      # foreground pixels of predicted mask

for img in X_test:
  mask_img = create_mask(model.predict(img))
  predictions.append(mask_img)
  fg_pixels.append(get_foreground(mask_img))

In [7]:
import pickle

# save foreground pixels
with open('drive/MyDrive/app/data/model_fg_pixels.pkl', 'wb') as f:
  pickle.dump(fg_pixels, f)

In [8]:
# load data from pickle file
#with open('drive/MyDrive/app/data/model_fg_pixels.pkl', 'rb') as f:
#  loaded_fg = pickle.load(f)